In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Parameters for model
max_len = 40
units = 128
learning_rate = 0.001
batch_size = 200
epoch = 10
temp = 0.5

In [3]:
def reading_of_data(file_name):
# Function to read input text file and to convert into lower letter
    text = open(file_name, 'r').read()
    return text.lower()

In [4]:
def Proccessing_for_Dataset(text):
# Function to convert text data into train and label fomrat
    unique_chars = list(set(text))
    len_unique_chars = len(unique_chars)

    input_chars = []
    output_char = []

    for i in range(0, len(text) - max_len, 2):
        input_chars.append(text[i:i+max_len])
        output_char.append(text[i+max_len])

    train_data = np.zeros((len(input_chars), max_len, len_unique_chars))
    target_data = np.zeros((len(input_chars), len_unique_chars))

    for i , each in enumerate(input_chars):
        for j, char in enumerate(each):
            train_data[i, j, unique_chars.index(char)] = 1
        target_data[i, unique_chars.index(output_char[i])] = 1
    return train_data, target_data, unique_chars, len_unique_chars

In [5]:
def RNN_LSTM_model(x, weight, bias, len_unique_chars):
# RNN model and predictions
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, len_unique_chars])
    x = tf.split(x, max_len, 0)

    cell = tf.contrib.rnn.BasicLSTMCell(units, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    prediction = tf.matmul(outputs[-1], weight) + bias
    return prediction

In [6]:
def Probability_of_sample(predicted):
# Function to extract indeces from probability array
    exp_pred = np.exp(predicted/temp)
    predicted = exp_pred / np.sum(exp_pred)
    prob = np.random.multinomial(1, predicted, 1)
    return prob

In [7]:
def Training_and_Generation_of_Text(train_data, target_data, unique_chars, len_unique_chars):
    
    # Initializing placeholder for input data
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    
    # Weight inintialization 
    weight = tf.Variable(tf.random_normal([units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    # Prediction using LSTM model
    prediction = RNN_LSTM_model(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    init_op = tf.global_variables_initializer()
    
    # start of tensorflow session
    sess = tf.Session()
    sess.run(init_op)

    num_batches = int(len(train_data)/batch_size)

    for i in range(epoch):
        print ("Epoch no:- {0} of {1} is generated below:".format(i+1, epoch))
        count = 0
        for _ in range(num_batches):
            train_batch, target_batch = train_data[count:count+batch_size], target_data[count:count+batch_size]
            count += batch_size
            sess.run([optimizer] ,feed_dict={x:train_batch, y:target_batch})

        # inserting a seed for generation
        seed = train_batch[:1:]

        # Seed is given as
        seed_chars = ''
        for each in seed[0]:
                seed_chars += unique_chars[np.where(each == max(each))[0][0]]
        print ("Seeding character is:", seed_chars)

        # Prediction of next 1000 characters
        for i in range(1000):
            if i > 0:
                remove_fist_char = seed[:,1:,:]
                seed = np.append(remove_fist_char, np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
            predicted = sess.run([prediction], feed_dict = {x:seed})
            predicted = np.asarray(predicted[0]).astype('float64')[0]
            probabilities = Probability_of_sample(predicted)
            predicted_chars = unique_chars[np.argmax(probabilities)]
            seed_chars += predicted_chars
        print ('Generated Text is given below after seed character:', seed_chars)
    sess.close()

In [8]:
ln = len(reading_of_data("warpeace_input.txt"))
ln

3196213

In [9]:
#Importing input text datset in the main function    
reduced_ln = int(ln*0.12)
data = reading_of_data('warpeace_input.txt')[:reduced_ln]
train_data, target_data, unique_chars, len_unique_chars = Proccessing_for_Dataset(data)
Training_and_Generation_of_Text(train_data, target_data, unique_chars, len_unique_chars)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch no:- 1 of 10 is generated below:
Seeding character is: oue que votre victoire
n'est pas des plu
Generated Text is given below after seed character: oue que votre victoire
n'est pas des plundes of of the porened the mand the wand ald of the rom mereand was a

Seeding character is: oue que votre victoire
n'est pas des plu
Generated Text is given below after seed character: oue que votre victoire
n'est pas des plussing word, and to me was he weited with a
prince andrew to one he with with a chees with a with the firen and had he still he was are the string or the the of when she were the ressian compartion when he were and soling in all as he say he was a centining the room was he went and the retime of the with a manche the his was aid and the disticarly from thembers and a she the was ness on firre looked to fiends of his down the reading and had he and he prince and without with whech and he was armided an the his on of the dinging of the fient and he was a contertion of the stance of the done of the without and whowe were he wishout the ment the with a crose the prince and ment the mone reserved of his was all were in my prince andrew sond his denision
comeas who were at his way a repleasing who was are capparing of the more firme the sen